# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
# imports
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import ollama

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
openai = OpenAI()

API key looks good so far


In [11]:
# here is the question; type over this to ask something new

def build_question(code_str):
    return f"Please explain what this code does and why:\n {code_str}"

In [8]:
print(build_question("import ollama"))

Please explain what this code does and why:
 import ollama


In [20]:
# Get gpt-4o-mini to answer, with streaming
system_prompt = "You are the most experienced Python developer / Tech Lead / System Architect who worked for major tech companies. \
You are also a teacher and a mentor for new developers. You are patient and explain everything thorowly. \
You are provided python code and your task is to explain it the best way possible, combining the thoery with examples. \
Respond in markdown."

def stream_coding_answer_gpt(code_str):
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": build_question(code_str)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [21]:
code_str = "yield from {book.get(\"author\") for book in books if book.get(\"author\")}"

In [22]:
stream_coding_answer_gpt(code_str)

Certainly! Let's break down the provided code step by step to understand what it does and why it is structured in this way.

### Code Breakdown

python
yield from {book.get("author") for book in books if book.get("author")}


#### 1. `yield from`
This is a Python keyword used within a generator function. It allows you to yield all values from another iterable (in this case, a set comprehension). When you use `yield from`, the generator will yield each value produced by the iterable one at a time.

**Theory in Practice:**
Using `yield from` is particularly useful when you have a nested generator, allowing for cleaner and more readable code. Instead of manually looping through the iterable and yielding each value, `yield from` takes care of that for you.

#### 2. Set Comprehension
Next, we have the portion of the code which is a set comprehension:

python
{book.get("author") for book in books if book.get("author")}


This part creates a set containing unique author names. Here’s how it works:

- **`for book in books`**: This iterates through each item in the collection called `books`. It is expected that `books` is some kind of iterable (like a list) that contains dictionaries, where each dictionary represents a book.

- **`book.get("author")`**: For each book (which is a dictionary), we attempt to retrieve the value associated with the key `"author"`. The `get` method is used here, which safely returns the value for the given key if it exists, and returns `None` if it doesn’t (instead of throwing a KeyError).

- **`if book.get("author")`**: This condition filters out any books where the author is not specified (i.e., where `book.get("author")` would be `None`). Only books with an actual author will have their author names included in the resulting set.

#### 3. The Overall Purpose
Putting it all together, the code is designed to create a generator that yields the unique authors from a list of books, omitting any `None` values. 

### Example
Let’s assume we have the following list of books:

python
books = [
    {"title": "Book One", "author": "Author A"},
    {"title": "Book Two", "author": "Author B"},
    {"title": "Book Three", "author": None},
    {"title": "Book Four", "author": "Author A"},
]


If we call the generator function that includes the code you provided, we would be yielding the following unique authors:

- **Output** (`Author A`, `Author B`)

Notice how "Author A" only appears once in the output irrespective of how many times they appear in the original list, because the authors are stored in a set, which inherently does not allow duplicates.

### Conclusion
In summary, the code:

- Uses a generator to yield unique authors from a list of book dictionaries.
- Safely retrieves the authors via the `get` method, avoiding potential errors by filtering out books without authors.
- Ensures that each author is yielded only once due to the nature of sets in Python.

This compact piece of code showcases Python’s powerful comprehension and generator capabilities in a practical manner. If you have any further questions or need clarifications on any part of this explanation, feel free to ask!

In [25]:
def build_messages(code_str):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": build_question(code_str)}
    ]

In [30]:
# Get Llama 3.2 to answer

def stream_coding_answer_ollama(code_str):
    stream = ollama.chat(model=MODEL_LLAMA, messages=build_messages(code_str), stream=True)
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk['message']['content'] or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [31]:
stream_coding_answer_ollama(code_str)

**Generator Expression: Understanding the Code**
=====================================================

The given code snippet is a Python generator expression that uses the `yield from` syntax. Let's break it down step by step:

python
yield from {book.get("author") for book in books if book.get("author")}


**Components of the Generator Expression**

1. **Collection Comprehension**: `{book.get("author") for book in books if book.get("author")}`
	* This part is a collection comprehension, which is similar to a list comprehension but creates an iterator instead.
	* It iterates over each `book` in the `books` collection and checks if it has a key called `"author"`.
	* If the condition is true (i.e., `book.get("author")` returns a value), the corresponding author's name is included in the iteration.

2. **`yield from` Statement**: `yield from { ... }`
	* This keyword introduces a new concept: the `yield from` statement.
	* When used with an iterator (like our collection comprehension), it delegates control to each inner item, which yields its own output instead of returning a single value.

**Putting It All Together**

The entire expression creates an iterator that yields author names from books that have authors. Here's what happens:

1. The generator expression iterates over `books`.
2. For each book, it checks if the `"author"` key exists and is not empty.
3. If a book has an author, its name is yielded by the collection comprehension.
4. However, instead of returning the list of authors directly (which would be equivalent to using a list comprehension), we use `yield from` to delegate control to each inner iterator.

**Example Walkthrough**
python
books = [
    {"author": "Author A", "title": "Book 1"},
    {"title": "Book 2"},  # no author
    {"author": "Author C", "title": "Book 3"}
]

for author in yield from {book.get("author") for book in books if book.get("author")}:
    print(author)

Output:


Author A
Author C


In summary, this code uses a generator expression with `yield from` to create an iterator that efficiently yields only the authors of books without actually loading all the data into memory. This makes it suitable for large datasets where memory usage is a concern.